# Generacion de Datasets utilizando RLHF para fine tunning

## Procedimiento

- Recoleccion de Datos: Obtener Informacion mediante documentos de textos seleccionables (no escaneados), mediante web, redes sociales, scrapping,etc
- Extraer Texto: Usa PyMuPDF para extraer texto de PDFs no escaneados, procesando en paralelo con ProcessPoolExecutor para manejar grandes volúmenes (>100, >10,000).
- Limpiar Datos: Elimina espacios múltiples y caracteres no deseados con expresiones regulares, asegurando texto coherente.
- Validar: Verifica que los fragmentos extraídos no estén vacíos y tengan una longitud mínima (por ejemplo, 50 caracteres).
- Generar Conversaciones: Divide el texto en párrafos, usa un modelo como meta-llama/Llama-3.2-1B-Instruct para generar diálogos conversacionales específicos al contenido, con al menos 4 intercambios por conversación, en formato {"messages": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}], "topic": "..."}.
- Estructurar Dataset: Guarda las conversaciones en archivos JSONL con dos columnas: "messages" y "topic".
Cargar y Subir: Carga el dataset con load_dataset("json", data_files="path/*.jsonl") y súbelo a un repositorio privado en Hugging Face Hub con push_to_hub("username/dataset_name", private=True).

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
%pip install --upgrade pymupdf nltk ollama


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

MIN_FRAGMENT_LENGTH = 500  # Longitud mínima de un fragmento
MAX_FRAGMENT_LENGTH = 2000  # Longitud máxima de un fragmento
REPEAT_THRESHOLD = 0.3  # Umbral para considerar un bloque como repetitivo

OLLAMA_MODEL = "llama3.1:8b"
TEMPERATURE = 0.7  # Equilibrio entre creatividad y coherencia
TOP_P = 0.9  # Filtrado de núcleo para diversidad
MIN_CONVERSATION_LENGTH = 3 

### Recoleccion de Datos

In [4]:
from pathlib import Path

# Verificar si la carpeta existe

folder_url = "/workspace/data/uploads"
folder = Path(folder_url)


if folder.exists() and folder.is_dir():
    print("Valid Folder")
    
# Obtener todos los archivos de la carpeta
files = [f for f in folder.rglob("*") if f.is_file()]

files
len(files)

Valid Folder


97

### Extraccion de Texto y Limpieza de Datos

In [5]:
import pymupdf
import re
import os
import hashlib
from collections import Counter

# Constantes
MIN_FRAGMENT_LENGTH = 500
MAX_FRAGMENT_LENGTH = 2000
REPEAT_THRESHOLD = 0.3
MIN_BLOCK_LENGTH = 30  # Reducido de 50 a 30 para incluir bloques más cortos

def clean_text(text, repeated_blocks=None):
    """Limpia el texto, eliminando caracteres repetitivos, texto redundante y caracteres no deseados."""
    # Eliminar caracteres de control no deseados (excepto \n)
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)
    # Eliminar patrones repetitivos como ----, ...., ****
    text = re.sub(r'([^\w\s])\1{2,}|\s*[.]{3,}\s*', '', text)
    # Normalizar espacios múltiples
    text = re.sub(r'[ \t]+', ' ', text)
    # Eliminar espacios al inicio y final de cada línea
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    # Filtrar líneas duplicadas, números, correos y contenido administrativo
    seen_lines = set()
    unique_lines = []
    for line in lines:
        if line not in seen_lines and \
           not re.match(r'^\d+$', line) and \
           not re.match(r'.*@(.*\.)+.*', line) and \
           not re.match(r'^(Tel|Fax|E-mail|www\.).*', line, re.IGNORECASE) and \
           (repeated_blocks is None or line not in repeated_blocks):
            unique_lines.append(line)
            seen_lines.add(line)
    return '\n'.join(unique_lines)

def extract_text_from_pdf(pdf_path):
    """
    Extrae texto de un PDF en una sola pasada, preservando el texto del inicio de la página,
    eliminando encabezados, pies de página y contenido irrelevante,
    dividiendo en fragmentos de 500 a 2000 caracteres con metadata corregida.
    """
    try:
        doc = pymupdf.open(pdf_path)
        total_pages = len(doc)
        chunks = []
        current_chunk = []  # Lista de (párrafo, página)
        current_chunk_length = 0
        filename = os.path.basename(pdf_path)
        block_counter = Counter()

        for page_number in range(1, total_pages + 1):
            page = doc[page_number - 1]
            page_height = page.rect.height
            blocks = page.get_text("blocks")
            page_text = []

            # Procesar bloques y filtrar encabezados/pies de página
            for block in blocks:
                text = block[4]
                y0, y1 = block[1], block[3]
                # Excluir pies de página (parte inferior de la página)
                if y1 > 0.95 * page_height:  # Relajado de 0.95 a 0.9
                    continue
                # Excluir encabezados solo si son repetitivos
                if y0 < 0.05 * page_height:  # Relajado de 0.05 a 0.1
                    block_counter[text] += 1
                    if block_counter[text] > total_pages * REPEAT_THRESHOLD:
                        continue
                if text and len(text) >= MIN_BLOCK_LENGTH:
                    block_counter[text] += 1
                    page_text.append((text, page_number))

            # Si no hay texto válido en la página, continuar
            if not page_text:
                continue

            # Acumular párrafos con su número de página
            for paragraph, page in page_text:
                current_chunk.append((paragraph, page))
                current_chunk_length += len(paragraph) + 2  # +2 por "\n\n"

                # Si el fragmento alcanza la longitud mínima, procesarlo
                if current_chunk_length >= MIN_FRAGMENT_LENGTH:
                    chunk_text = "\n\n".join(p for p, _ in current_chunk)
                    cleaned_chunk = clean_text(chunk_text, None)
                    cleaned_length = len(cleaned_chunk)

                    # Obtener el rango de páginas del fragmento
                    page_numbers = sorted(set(page for _, page in current_chunk))
                    start_page = page_numbers[0] if page_numbers else page_number
                    end_page = page_numbers[-1] if page_numbers else page_number

                    # Dividir fragmentos largos
                    while cleaned_length > MAX_FRAGMENT_LENGTH:
                        sub_chunk = cleaned_chunk[:MAX_FRAGMENT_LENGTH]
                        last_paragraph_end = sub_chunk.rfind("\n\n")
                        if last_paragraph_end == -1:
                            last_paragraph_end = MAX_FRAGMENT_LENGTH
                        chunk_to_add = cleaned_chunk[:last_paragraph_end].strip()

                        # Calcular el número de páginas para el subfragmento
                        chars_so_far = 0
                        sub_chunk_pages = []
                        for paragraph, page in current_chunk:
                            chars_so_far += len(paragraph) + 2
                            if chars_so_far <= last_paragraph_end:
                                sub_chunk_pages.append(page)
                            else:
                                break
                        sub_start_page = min(sub_chunk_pages) if sub_chunk_pages else page_number
                        sub_end_page = max(sub_chunk_pages) if sub_chunk_pages else page_number

                        metadata = (
                            f"# FILENAME: {filename} | CHARACTERS: {len(chunk_to_add)} | "
                            f"PAGES: {sub_start_page}-{sub_end_page}/{total_pages}\n\n"
                        )
                        chunks.append((metadata + chunk_to_add, hashlib.md5(chunk_to_add.encode()).hexdigest()))
                        cleaned_chunk = cleaned_chunk[last_paragraph_end:].strip()
                        cleaned_length = len(cleaned_chunk)

                        # Actualizar current_chunk para los párrafos restantes
                        remaining_chunk = []
                        chars_so_far = 0
                        for paragraph, page in current_chunk:
                            chars_so_far += len(paragraph) + 2
                            if chars_so_far > last_paragraph_end:
                                remaining_chunk.append((paragraph, page))
                        current_chunk = remaining_chunk
                        current_chunk_length = cleaned_length
                        page_numbers = sorted(set(page for _, page in current_chunk))
                        start_page = page_numbers[0] if page_numbers else page_number

                    # Añadir el fragmento completo
                    if cleaned_length >= MIN_FRAGMENT_LENGTH:
                        metadata = (
                            f"# FILENAME: {filename} | CHARACTERS: {cleaned_length} | "
                            f"PAGES: {start_page}-{end_page}/{total_pages}\n\n"
                        )
                        chunks.append((metadata + cleaned_chunk, hashlib.md5(cleaned_chunk.encode()).hexdigest()))
                        current_chunk = []
                        current_chunk_length = 0

                    else:
                        current_chunk = [(cleaned_chunk, page_numbers[-1])] if page_numbers else []
                        current_chunk_length = cleaned_length

        # Añadir el fragmento final si cumple con la longitud mínima
        if current_chunk and current_chunk_length >= MIN_FRAGMENT_LENGTH:
            chunk_text = "\n\n".join(p for p, _ in current_chunk)
            cleaned_chunk = clean_text(chunk_text, None)
            cleaned_length = len(cleaned_chunk)
            if cleaned_length >= MIN_FRAGMENT_LENGTH:
                page_numbers = sorted(set(page for _, page in current_chunk))
                start_page = page_numbers[0] if page_numbers else total_pages
                end_page = page_numbers[-1] if page_numbers else total_pages
                metadata = (
                    f"# FILENAME: {filename} | CHARACTERS: {cleaned_length} | "
                    f"PAGES: {start_page}-{end_page}/{total_pages}\n\n"
                )
                chunks.append((metadata + cleaned_chunk, hashlib.md5(cleaned_chunk.encode()).hexdigest()))

        doc.close()

        # Filtrar bloques repetitivos y duplicados
        repeated_blocks = {text for text, count in block_counter.items() if count > total_pages * REPEAT_THRESHOLD}
        final_chunks = []
        seen_hashes = set()

        for chunk, chunk_hash in chunks:
            chunk_text = '\n'.join(line for line in chunk.splitlines() if not line.startswith('#'))
            cleaned_chunk = clean_text(chunk_text, repeated_blocks)
            if len(cleaned_chunk) >= MIN_FRAGMENT_LENGTH and chunk_hash not in seen_hashes:
                # Actualizar la longitud en la metadata después de la limpieza final
                metadata_lines = chunk.splitlines()[0]
                metadata = re.sub(r'CHARACTERS: \d+', f'CHARACTERS: {len(cleaned_chunk)}', metadata_lines)
                final_chunks.append(f"{metadata}\n\n{cleaned_chunk}")
                seen_hashes.add(chunk_hash)

        return final_chunks if final_chunks else None
    except Exception as e:
        print(f"Error procesando {pdf_path}: {e}")
        return None

### Generar Conversaciones

In [6]:
from ollama import AsyncClient
from pydantic import BaseModel, Field
from typing import List


# Definir el esquema Pydantic para la salida estructurada
class Message(BaseModel):
    role: str = Field(..., pattern="^(system|user|assistant)$")
    content: str

class Conversation(BaseModel):
    messages: List[Message] = Field(..., min_items=MIN_CONVERSATION_LENGTH)
    topic: str

ollama_client = AsyncClient()

async def generate_conversation(fragment, temperature=TEMPERATURE, top_p=TOP_P):
    """
    Genera una conversación estructurada en formato JSON usando Ollama, basada en un fragmento de texto.
    Optimizado para múltiples iteraciones en la generación de datasets para fine-tuning.
    """
    if not fragment or len(fragment) < MIN_FRAGMENT_LENGTH:
        print(f"Error: Fragmento demasiado corto ({len(fragment)} caracteres).")
        return None

    # Extraer el texto sin la metadata (líneas que comienzan con '#')
    fragment_content = '\n'.join(line for line in fragment.splitlines() if not line.startswith('#')).strip()
    if len(fragment_content) < MIN_FRAGMENT_LENGTH:
        print(f"Error: Contenido útil del fragmento demasiado corto ({len(fragment_content)} caracteres).")
        return None

    # Prompt optimizado para múltiples iteraciones
    prompt = f"""
<context>
{fragment_content}
</context>

<instructions>
Basándote únicamente en el texto dentro de <context>...</context>, genera una conversación estructurada entre un usuario y un asistente con las siguientes características:

1. **Estructura**:
   - Genera de {MIN_CONVERSATION_LENGTH} a 8 intercambios (parejas usuario-asistente). Usa 2 intercambios para fragmentos con poco contexto (por ejemplo, listas o títulos cortos) y de 4 a 8 para fragmentos detallados.
   - La conversación debe comenzar con una pregunta del usuario que refleje un interés específico en el contenido (por ejemplo, explorar regulaciones bancarias).
2. **Preguntas del usuario**:
   - Deben ser específicas, relevantes y basadas exclusivamente en el fragmento.
   - Evita preguntas genéricas como “¿Cuál es el tema?”, “¿Qué organización se menciona?”, “¿Qué menciona en este fragmento?”  o “¿Cuál es el propósito?”, etc....
   - Incluye una mezcla de:
     - Preguntas factuales (por ejemplo, “¿Qué requisitos se relajan durante el periodo de transición?”).
     - Frases incompletas para completar (por ejemplo, “El periodo de transición para el método IRB básico es...”).
     - Preguntas de análisis o inferencia (por ejemplo, “¿Por qué es importante un periodo de transición para los bancos?”).
   - Las preguntas deben formar un flujo conversacional lógico, donde cada una se base en la respuesta anterior.
3. **Respuestas del asistente**:
   - Deben ser claras, concisas y basadas únicamente en el fragmento.
   - Deben responder directamente a la pregunta con un tono profesional y conversacional.
4. **Tema**:
   - Resume el tema principal del fragmento en el campo `topic` (máximo 50 caracteres).
5. **Formato**:
   - Retorna un objeto JSON que cumpla con el esquema de `Conversation`, con un campo `messages` que contenga una lista de objetos `Message` (con `role` = "user" o "assistant" y `content`) y un campo `topic`.

<example>
**Fragmento**:
```
(A)
PERIODO DE TRANSICIÓN RELATIVO A LA IMPLEMENTACIÓN GENERAL DEL ACUERDO
215.
El Nuevo Acuerdo será aplicable a todos los bancos internacionalmente activos en cada nivel del grupo bancario. Aquellos países en los que la subconsolidación no es actualmente un requisito, tendrán un periodo de transición de tres años, a partir de la fecha de implementación, para aplicarla.
(B)
PERIODO DE TRANSICIÓN RELATIVO AL MÉTODO FUNDADO EN LA CALIFICACIÓN INTERNA
216.
El Comité reconoce que un respeto completo e inmediato de ciertos requisitos relacionados con datos podría no ser posible...
```

**Salida esperada**:
```json
{{
    "messages": [
        {{"role": "user", "content": "¿Qué bancos deben cumplir con este acuerdo?"}},
        {{"role": "assistant", "content": "Todos los bancos internacionalmente activos en cada nivel del grupo bancario."}},
        {{"role": "user", "content": "¿Por qué es necesaria la subconsolidación?"}},
        {{"role": "assistant", "content": "Para garantizar que los riesgos se gestionen adecuadamente en todos los niveles del grupo bancario."}},
        {{"role": "user", "content": "¿Qué requisitos del método IRB se relajan durante el periodo de transición?"}},
        {{"role": "assistant", "content": "Ciertos requisitos relacionados con datos para exposiciones empresariales, bancarias, soberanas y al detalle."}}
    ],
    "topic": "Periodo de transición del Acuerdo de Basilea"
}}
```

<notes>
- Evita repetir preguntas o respuestas similares para mantener la diversidad en múltiples iteraciones.
- Asegúrate de que la conversación sea útil para entrenar modelos conversacionales, reflejando interacciones humanas verosímiles.
- Si el fragmento es insuficiente para una conversación completa, genera un solo intercambio (1 pregunta y 1 respuesta).
- no es necesario que la respuesta seamas de 2 intercambios, puede ser de un intercambio, siempre y cuando la respuesta del asistente sea lo mas detallada posible.
- siempre cubre todo el contexto generando conversaciones, no te saltes ningun parrafo, usalo para la generacion de conversaciones.
</notes>
    """

    try:
        # Llama a Ollama con roles system y user
        response = await ollama_client.chat(
            model=OLLAMA_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "Eres un experto en el dominio del fragmento proporcionado (por ejemplo, regulaciones financieras, riesgos financieros, finanzas). Genera respuestas claras, concisas y basadas únicamente en el fragmento, manteniendo un tono profesional y conversacional, se especifico y detalla una respuestas en el intercambio de asistente."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            options={
                "temperature": temperature,
                "top_p": top_p,
            },
            format=Conversation.model_json_schema()  # Especifica el esquema JSON
        )
        conversation = Conversation.model_validate_json(response.message.content)
        # Validar que la conversación tenga al menos un intercambio completo
        if len(conversation.messages) < 2:
            print(f"Error: Conversación inválida, número de mensajes insuficiente: {len(conversation.messages)}")
            return None
        # Si el número de mensajes es impar, eliminar el último (asumiendo que es del usuario)
        if len(conversation.messages) % 2 != 0 and len(conversation.messages) > 2:
            conversation.messages = conversation.messages[:-1]
        return conversation
    except Exception as e:
        print(f"Error generando conversación con Ollama: {e}")
        return None

In [ ]:
import json
from tqdm.asyncio import tqdm_asyncio
async def process_pdf(index, pdf_path, output_dir):
    """Procesa un PDF, genera conversaciones y las guarda en JSONL."""
    try:
        
        pages_text = extract_text_from_pdf(pdf_path)

        if not pages_text:
            return
    
        jsonl_file = os.path.join(output_dir, f"pdf_{index:04d}.jsonl")
        with open(jsonl_file, "w", encoding="utf-8") as f:
            tasks = [generate_conversation(fragment) for fragment in pages_text if len(fragment) > 20]
    
            conversations = await tqdm_asyncio.gather(*tasks,  desc=f"Procesando Fragmentos para indice {index}...")
    
            if conversations:
                output = [messages.model_dump() for messages in conversations if messages is not None]
                output_str = "\n".join(json.dumps(messages, ensure_ascii=False) for messages in output)
                f.write(output_str + "\n")
    except Exception as e:
        print(f"Error el pdf {pdf_path}: {e}")
    

In [9]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import asyncio
import os
from pathlib import Path

def process_pdf_wrapper(args):
    index, pdf_path, output_dir = args
    # Ejecutar la función asíncrona process_pdf dentro de un bucle de eventos
    asyncio.run(process_pdf(index, pdf_path, output_dir))

def generate():
    pdf_files = files
    output_dir="outputs"
    output_folder_path=Path(output_dir)
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    num_workers = int(min(multiprocessing.cpu_count() * 0.8,len(pdf_files)))
    with ProcessPoolExecutor(max_workers=15) as executor:
        list(tqdm(executor.map(process_pdf_wrapper, [(i, p, output_dir) for i, p in enumerate(pdf_files)]), total=len(pdf_files)))

In [9]:
%%time
generate()

Procesando Fragmentos para indice 16...:   0%|          | 0/56 [00:00<?, ?it/s]

Error generando conversación con Ollama: Event loop is closed


Procesando Fragmentos para indice 17...:   0%|          | 0/150 [00:00<?, ?it/s]

Error generando conversación con Ollama: Event loop is closed


Procesando Fragmentos para indice 18...:   0%|          | 0/20 [00:00<?, ?it/s]

...

Procesando Fragmentos para indice 54...:   0%|          | 0/12 [00:00<?, ?it/s]

Error generando conversación con Ollama: Event loop is closed



Procesando Fragmentos para indice 65...:   0%|          | 0/6 [00:00<?, ?it/s]

Procesando Fragmentos para indice 68...:   0%|          | 0/28 [00:00<?, ?it/s]Process ForkProcess-5:
Process ForkProcess-4:
Process ForkProcess-7:
Process ForkProcess-10:
Task was destroyed but it is pending!
task: <Task pending name='Task-448' coro=<<async_generator_athrow without __name__>()>>
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/.local/share/uv/python/cpython-3.11-linux-x86_64-gnu/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Traceback (most recent call last):
  File "/root/.local/share/uv/python/cpython-3.11-linux-x86_64-gnu/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/.local/share/uv/python/cpython-3.11-lin

CPU times: user 3.27 s, sys: 1.68 s, total: 4.95 s
Wall time: 2h 49min 22s



KeyboardInterrupt


KeyboardInterrupt



### Saving

In [11]:
from datasets import load_dataset


dataset = load_dataset("json", data_files="./outputs/*.jsonl")

dataset


Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages', 'topic'],
        num_rows: 664
    })
})

In [10]:
from huggingface_hub import login
login()

ERROR! Session/line number was not unique in database. History logging moved to new session 16


In [20]:
dataset.push_to_hub("jeanmcm/b_risks")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########|  270kB /  270kB            

README.md:   0%|          | 0.00/376 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jeanmcm/b_risks/commit/13e6a6ad8f5d4eb1efc3bf36fe4e0b58c6e237fb', commit_message='Upload dataset', commit_description='', oid='13e6a6ad8f5d4eb1efc3bf36fe4e0b58c6e237fb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jeanmcm/b_risks', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jeanmcm/b_risks'), pr_revision=None, pr_num=None)

# Testing RAG with Open WebUI 

In [36]:
import requests
import json
import sys
import time
from IPython.display import display, clear_output, HTML

url = "https://vwlppjjfa98c9x-8080.proxy.runpod.net"
api_key ="sk-05568562f28844fe997cadf960a346cd"

messages =  [{"role": "user", "content": "Que es el Riesgo Financiero?"}]

try:
    # Realizar la solicitud con stream=True
    with requests.post(f"{url}/api/chat/completions", stream=True,headers={
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    },json={
      "model":"bosft-riesgos-rag-model",
      "messages":messages,
      "stream":True
      }) as response:
        response.raise_for_status()
        # Variable para almacenar la salida acumulada
        accumulated_output = ""

        # Iterar sobre las líneas de la respuesta
        for line in response.iter_lines():
            if line:
                # Decodificar la línea
                decoded_line = line.decode('utf-8').strip()
                # Si la línea comienza con "data:", extraer el contenido
                if decoded_line.startswith("data:"):
                    decoded_line = decoded_line[5:].strip()  # Quitar "data: "

                # Ignorar líneas vacías o marcadores de fin como "[DONE]"
                if not decoded_line or decoded_line == "[DONE]" :
                    continue
                
                
                try:
                    # Parsear si es JSON
                    data = json.loads(decoded_line)
                    if "choices" not in data: continue
                    
                    delta = data['choices'][0]['delta']
                    if "content" in delta: new_data = delta['content']
                except json.JSONDecodeError:
                    # Si no es JSON, usar la línea como texto
                    new_data = decoded_line

                # Acumular y mostrar la salida dinámicamente
                if new_data:
                    accumulated_output += new_data
                    # Limpiar la salida anterior y mostrar la nueva
                    clear_output(wait=True)
                    display(HTML(f"<b>Respuesta en streaming:</b> {accumulated_output}"))
                    time.sleep(0.1)  # Pequeña pausa para visibilidad
                    

except requests.exceptions.RequestException as e:
    print(f"\nError en la solicitud: {e}")

# Testing with Flowise